In [315]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import json
from tqdm import tqdm

# Finam

##  Получение Российских акций

In [126]:
url = "https://www.finam.ru/quotes/stocks/russia/"
driver = webdriver.Chrome()
driver.get(url)

In [132]:
news_section = driver.find_element(By.ID, 'finfin-local-plugin-quote-table-table-more-container')
news_section.click()

In [133]:
table = driver.find_element(By.ID, 'finfin-local-plugin-quote-table-table-table')

In [134]:
list_of_instr = table.find_elements(By.TAG_NAME, 'tr')

In [135]:
len(list_of_instr)

301

In [143]:
list_id = []
list_link = []
list_name = []

for i in range(1,301):
    list_id.append(list_of_instr[i].get_attribute('data-quoteid'))
    list_link.append(list_of_instr[i].find_element(By.TAG_NAME, 'a').get_attribute('href'))
    list_name.append(list_of_instr[i].find_element(By.TAG_NAME, 'td').text)


In [149]:
data_stock = {
    'em': list_id,
    'name': list_name,
    'link': list_link
}

df_stock = pd.DataFrame(data_stock)
df_stock.to_parquet('finam_ru_stock_300.parquet')
df_stock

,em,name,link
0,29,Аэрофлот,https://www.finam.ru/quote/moex/aflt/
1,19043,ВТБ ао,https://www.finam.ru/quote/moex/vtbr/
2,16842,ГАЗПРОМ ао,https://www.finam.ru/quote/moex/gazp/
3,795,ГМКНорНик,https://www.finam.ru/quote/moex/gmkn/
4,8,ЛУКОЙЛ,https://www.finam.ru/quote/moex/lkoh/
...,...,...,...
295,4271197,Новошип ао,https://www.finam.ru/quote/moex/nomp/
296,4492039,ТКЗамещОбл (ЗПИФ Тинькофф Замещающие облиг),https://www.finam.ru/quote/moex/ru000a106g80/
297,499245,СоврАрБиз3 (ПИФ Современ Арендный бизнес 3),https://www.finam.ru/quote/moex/ru000a0zz5r2/
298,2396436,BCSB ETF (БКС Облигации повыш доход),https://www.finam.ru/quote/moex/bcsb/


In [160]:
driver.quit()

## Получение тех информации по акциям

In [317]:
driver = webdriver.Chrome()

Перебираем все акции  и получаем их тикеры и инструменты прожажи

In [ ]:
list_ticker = []
list_market_id = []
list_market_name = []

In [345]:
for url_stock in tqdm(list_link):
    driver.get(url_stock) # переход на страницу акции
    
    # поиск экспорта
    export = driver.find_element(By.ID, 'finfin-local-plugin-quote-item-item-tabs-content-tech')
    
    try:
        export_page = export.find_elements(By.TAG_NAME, 'a')[1].get_attribute('href')
    except:
        list_ticker.append(None)
        list_market_id.append(None)
        list_market_name.append(None)
        continue
    
    driver.get(export_page) # переход на страницу экспорта 

    # поиск данных акции
    info = driver.find_element(By.ID, 'content-block')
    script_tag = info.find_element(By.TAG_NAME, 'script')
    script = script_tag.get_attribute('text')
    
    # отчистка
    string_info = script.split("Finam.")[4].replace('\n', '')
    test_string = string_info.replace('IssuerProfile.Main.issue = ', '').replace(', }', '}').replace(',}', '}').replace('&quot', '')
    res = json.loads(test_string[:-1]) # получение словаря 

    list_ticker.append(res['quote']['code'])
    list_market_id.append(res['quote']['market']['id'])
    list_market_name.append(res['quote']['market']['title'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [04:32<00:00,  3.68s/it]


In [346]:
data_stock = {
    'em': list_id,
    '_company_name': list_name,
    'link': list_link,
    'ticker': list_ticker,
    'market_id': list_market_id,
    'market_name': list_market_name
}

df_stock = pd.DataFrame(data_stock)
df_stock.to_parquet('finam_ru_stock_300.parquet')
df_stock.dropna()

,em,_company_name,link,ticker,market_id,market_name
0,29,Аэрофлот,https://www.finam.ru/quote/moex/aflt/,AFLT,1.0,МосБиржа акции
1,19043,ВТБ ао,https://www.finam.ru/quote/moex/vtbr/,VTBR,1.0,МосБиржа акции
2,16842,ГАЗПРОМ ао,https://www.finam.ru/quote/moex/gazp/,GAZP,1.0,МосБиржа акции
3,795,ГМКНорНик,https://www.finam.ru/quote/moex/gmkn/,GMKN,1.0,МосБиржа акции
4,8,ЛУКОЙЛ,https://www.finam.ru/quote/moex/lkoh/,LKOH,1.0,МосБиржа акции
...,...,...,...,...,...,...
295,4271197,Новошип ао,https://www.finam.ru/quote/moex/nomp/,MOEX.NOMP:MTQR,1.0,МосБиржа акции
296,4492039,ТКЗамещОбл (ЗПИФ Тинькофф Замещающие облиг),https://www.finam.ru/quote/moex/ru000a106g80/,MOEX.RU000A106G80:TQIF,29.0,МосБиржа пифы
297,499245,СоврАрБиз3 (ПИФ Современ Арендный бизнес 3),https://www.finam.ru/quote/moex/ru000a0zz5r2/,RU000A0ZZ5R2,29.0,МосБиржа пифы
298,2396436,BCSB ETF (БКС Облигации повыш доход),https://www.finam.ru/quote/moex/bcsb/,MOEX.BCSB:TQTF,515.0,Мосбиржа ETF


In [347]:
driver.quit()

## Получение списка бирж 

In [287]:
test_string = script.split(";")[2].replace('\n', '').replace('Finam.IssuerProfile.Main.setMarkets([', '').replace(', }', '}').replace(',}', '}').replace('])', '')

In [291]:
list_shops = test_string.split('}, ')

for i in range(len(list_shops) - 1):
    list_shops[i] = (list_shops[i] + '}')

for i in range(len(list_shops)):
    list_shops[i] = list_shops[i].replace('\'', '\"')

    dict_split = list_shops[i].split(' ')
    dict_split[1] = ' \"' + dict_split[1][:-1] + '\"' + ': '
    dict_split[3] = ' \"' + dict_split[3][:-1] + '\"' + ': '
    list_shops[i] = json.loads(''.join(dict_split))


In [300]:
list_value = []
list_title = []

for i in list_shops:
    list_value.append(i['value'])
    list_title.append(i['title'])

data_shops = {
    'value': list_value,
    'title': list_title
    }

shops = pd.DataFrame(data_shops)
shops

,value,title
0,631,АкцииГермании
1,632,АкцииЕвропы
2,200,МосБиржатоп
3,1,МосБиржаакции
4,14,МосБиржафьючерсы
5,41,Курсрубля
6,45,МосБиржавалютныйрынок
7,2,МосБиржаоблигации
8,12,МосБиржавнесписочныеоблигации
9,29,МосБиржапифы


In [302]:
shops.to_parquet('ru_market.parquet')